In [1]:
import tkinter as tk
import random

# Initialize the game grid
def initialize_grid():
    grid = [[0] * 4 for _ in range(4)]
    add_new_tile(grid)
    add_new_tile(grid)
    return grid

def add_new_tile(grid):
    empty_cells = [(r, c) for r in range(4) for c in range(4) if grid[r][c] == 0]
    if empty_cells:
        r, c = random.choice(empty_cells)
        grid[r][c] = 2 if random.random() < 0.9 else 4

def slide_row_left(row):
    new_row = [i for i in row if i != 0]
    for i in range(len(new_row) - 1):
        if new_row[i] == new_row[i + 1]:
            new_row[i] *= 2
            new_row[i + 1] = 0
    new_row = [i for i in new_row if i != 0]
    return new_row + [0] * (len(row) - len(new_row))

def move_left(grid):
    return [slide_row_left(row) for row in grid]

def transpose(grid):
    return [list(row) for row in zip(*grid)]

def reverse(grid):
    return [row[::-1] for row in grid]

def move_right(grid):
    return reverse(move_left(reverse(grid)))

def move_up(grid):
    return transpose(move_left(transpose(grid)))

def move_down(grid):
    return transpose(move_right(transpose(grid)))

def game_over(grid):
    for row in grid:
        if 0 in row:
            return False
    for row in grid + transpose(grid):
        for i in range(3):
            if row[i] == row[i + 1]:
                return False
    return True

class Game2048:
    def __init__(self, master):
        self.master = master
        self.master.title("2048 Game")
        self.grid = initialize_grid()

        self.size = 4
        self.cell_size = 100
        self.padding = 10
        self.canvas = tk.Canvas(self.master, width=self.size * self.cell_size, height=self.size * self.cell_size, bg="#bbada0")
        self.canvas.pack()

        self.draw_grid()
        self.master.bind("<Key>", self.key_press)

    def draw_grid(self):
        self.canvas.delete("all")
        for r in range(self.size):
            for c in range(self.size):
                value = self.grid[r][c]
                if value == 0:
                    color = "#cdc1b4"
                else:
                    color = self.get_color(value)
                x0 = c * self.cell_size + self.padding
                y0 = r * self.cell_size + self.padding
                x1 = (c + 1) * self.cell_size - self.padding
                y1 = (r + 1) * self.cell_size - self.padding
                self.canvas.create_rectangle(x0, y0, x1, y1, fill=color, outline="")
                if value != 0:
                    self.canvas.create_text((x0 + x1) / 2, (y0 + y1) / 2, text=str(value), font=("Helvetica", 24, "bold"), fill="#776e65")

    def get_color(self, value):
        colors = {
            2: "#eee4da", 4: "#ede0c8", 8: "#f2b179", 16: "#f59563",
            32: "#f67c5f", 64: "#f65e3b", 128: "#edcf72", 256: "#edcc61",
            512: "#edc850", 1024: "#edc53f", 2048: "#edc22e"
        }
        return colors.get(value, "#3c3a32")

    def key_press(self, event):
        if event.keysym == 'Up':
            self.grid = move_up(self.grid)
        elif event.keysym == 'Down':
            self.grid = move_down(self.grid)
        elif event.keysym == 'Left':
            self.grid = move_left(self.grid)
        elif event.keysym == 'Right':
            self.grid = move_right(self.grid)
        else:
            return

        add_new_tile(self.grid)
        self.draw_grid()

        if game_over(self.grid):
            self.canvas.create_text(self.size * self.cell_size / 2, self.size * self.cell_size / 2, text="Game Over", font=("Helvetica", 48, "bold"), fill="#776e65")

if __name__ == "__main__":
    root = tk.Tk()
    game = Game2048(root)
    root.mainloop()
